In [1]:
import sqlite3


In [2]:
conn = sqlite3.connect('../data/preproc/preproc.sqlite3')

In [3]:
query = """
select id, name, status, type, error, meta from task
"""

c = conn.cursor()
c.execute(query)

names = c.fetchall()

In [4]:
import pandas as pd
import numpy as np
import json
def extract_platform(name, meta):
    if 'task' in meta:
        return meta['task'].split('_')[1]
    if 'platform' in meta:
        return meta['platform']
    tokens = name.split('_')
    if len(tokens) == 2 and \
        tokens[1].startwith('GPL'):
            return tokens[1]
    return np.nan

def extract_series(name, meta):
    if 'task' in meta:
        return meta['task'].split('_')[0]
    if 'accession' in meta:
        return meta['accession']
    tokens = name.split('_')
    if len(tokens) == 2 and \
        tokens[0].startwith('GSE'):
            return tokens[0]
    return np.nan


In [5]:
AGILENT_PLATFORMS = ['GPL4133', 'GPL6480', 'GPL15849', 'GPL1708', 'GPL887', 'GPL4091', 'GPL9128', 'GPL7264', 'GPL11387',
             'GPL8687', 'GPL6848', 'GPL2879', 'GPL5477', 'GPL8841', 'GPL10123', 'GPL4093', 'GPL11386', 'GPL10806',
             'GPL8269', 'GPL10150', 'GPL8583', 'GPL15931', 'GPL4126', 'GPL10152', 'GPL16050', 'GPL2567', 'GPL9053',
             'GPL14550', 'GPL5325', 'GPL10808', 'GPL13691', 'GPL9075', 'GPL8736', 'GPL885', 'GPL9777', 'GPL7504',
             'GPL8693', 'GPL2873', 'GPL17077', 'GPL10734', 'GPL13953', 'GPL13607', 'GPL13685', 'GPL7015', 'GPL15560',
             'GPL18623', 'GPL10481', 'GPL16280', 'GPL8737', 'GPL11068']

In [6]:
FRMA_PLATFORMS = ['GPL' + str(x) for x in [
    570, 571, 96,
    1261,  # organism Mus musculus
    # 6244, getting strange errors. data in matrix
    # 6246, # no frma package (?). read.affybatch: The affy package can process data from the Gene ST 1.x series of arrays,
    # but you should consider using either the oligo or xps packages, which are specifically
    # designed for these arrays.
    # there is no package called ‘mogene10stv1frmavecs’

    # 90, # organism Saccharomyces cerevisiae
    3921,
    # 5175, # no frma package (?). read.affybatch: The affy package is not designed for this array type. 
    # Please use either the oligo or xps package.
    # 5188, getting strange errors. data in matrix
    8321  # organism Mus musculus
]]

In [7]:
def extract_method(platform):
    if platform in FRMA_PLATFORMS:
        return 'affymetrix'
    
    if platform in AGILENT_PLATFORMS:
        return 'agilent'
    
    return np.nan

    
data = (
    pd.DataFrame
    .from_records(names, columns=['id', 'name', 'status', 'type', 'error', 'meta'])
    .set_index('id')
    .assign(meta=lambda d: d.meta.map(json.loads))
    .assign(platform=lambda d: d['name'].combine(d['meta'], func=extract_platform))
    .assign(series=lambda d: d['name'].combine(d['meta'], func=extract_series))
    .assign(method=lambda d: d['platform'].map(extract_method))
)
print(data.shape)
data[:10]

(19320, 8)


,name,status,type,error,meta,platform,series,method
id,,,,,,,,
1,GSE53224_GPL570,done,preproc,None,"{'accession': 'GSE53224', 'platform': 'GPL570'...",GPL570,GSE53224,affymetrix
2,GSE53183_GPL570,done,preproc,None,"{'accession': 'GSE53183', 'platform': 'GPL570'...",GPL570,GSE53183,affymetrix
3,GSE53157_GPL570,done,preproc,None,"{'accession': 'GSE53157', 'platform': 'GPL570'...",GPL570,GSE53157,affymetrix
4,GSE65721_GPL5082,skip,preproc,None,"{'accession': 'GSE65721', 'platform': 'GPL5082...",GPL5082,GSE65721,NaN
5,GSE65721_GPL570,done,preproc,None,"{'accession': 'GSE65721', 'platform': 'GPL570'...",GPL570,GSE65721,affymetrix
6,GSE53092_GPL570,done,preproc,None,"{'accession': 'GSE53092', 'platform': 'GPL570'...",GPL570,GSE53092,affymetrix
7,GSE65707_GPL570,done,preproc,None,"{'accession': 'GSE65707', 'platform': 'GPL570'...",GPL570,GSE65707,affymetrix
8,GSE53059_GPL570,done,preproc,None,"{'accession': 'GSE53059', 'platform': 'GPL570'...",GPL570,GSE53059,affymetrix
9,GSE53046_GPL570,done,preproc,None,"{'accession': 'GSE53046', 'platform': 'GPL570'...",GPL570,GSE53046,affymetrix


In [12]:
data.type.unique()

array(['preproc', 'convert-merge', 'manual'], dtype=object)

In [21]:
(
    data
    .query('method == "affymetrix"' 
           'and status == "done" and type == "convert-merge"')
    [['series', 'platform']]
    .drop_duplicates()
    .to_pickle('../data/preproc/series.converted.res1.pickle')
)

In [8]:
skin_age_series = [
    'GSE11622', 'GSE18098', 'GSE18876', 'GSE22396', 'GSE27949',
       'GSE27951', 'GSE34460', 'GSE34535', 'GSE34536', 'GSE35411',
       'GSE37223', 'GSE37689', 'GSE42432', 'GSE42902', 'GSE45516',
       'GSE47162', 'GSE47511', 'GSE47512', 'GSE47513', 'GSE51518',
       'GSE51519', 'GSE53424', 'GSE53425', 'GSE53426', 'GSE53431',
       'GSE54638', 'GSE55664', 'GSE57225', 'GSE58121', 'GSE62354',
       'GSE66511', 'GSE67098', 'GSE6710', 'GSE67762'
]

In [10]:
data[data.series.isin(skin_age_series)].query('status == "done"')

,name,status,type,error,meta,platform,series,method
id,,,,,,,,
120,GSE67098_GPL570,done,preproc,None,"{'samples': ['GSM1638783', 'GSM1638784', 'GSM1...",GPL570,GSE67098,affymetrix
781,GSE45516_GPL570,done,preproc,None,"{'samples': ['GSM1106065', 'GSM1106066', 'GSM1...",GPL570,GSE45516,affymetrix
1778,GSE27951_GPL570,done,preproc,None,"{'samples': ['GSM691122', 'GSM691125', 'GSM691...",GPL570,GSE27951,affymetrix
1925,GSE27949_GPL570,done,preproc,None,"{'samples': ['GSM691122', 'GSM691125', 'GSM691...",GPL570,GSE27949,affymetrix
4212,GSE11622_GPL570,done,preproc,None,"{'samples': ['GSM294856', 'GSM294857', 'GSM294...",GPL570,GSE11622,affymetrix
5416,GSE6710_GPL96,done,preproc,None,"{'samples': ['GSM154768', 'GSM154769', 'GSM154...",GPL96,GSE6710,affymetrix
12278,GSE18098_GPL2879,done,preproc,None,"{'samples': ['GSM452353', 'GSM452354', 'GSM452...",GPL2879,GSE18098,agilent
12963,GSE18098_GPL5477,done,preproc,None,"{'samples': ['GSM452366', 'GSM452367', 'GSM452...",GPL5477,GSE18098,agilent
13941,GSE67098_GPL570_convert,done,convert-merge,None,"{'samples': ['GSM1638783', 'GSM1638784', 'GSM1...",GPL570,GSE67098,affymetrix


In [8]:
import pymongo
db = pymongo.MongoClient().scraper_meta

series_accessions = data.series.dropna().unique().tolist()

samples = (
    pd.DataFrame(list(db.series.find({'accession': {'$in': series_accessions}}, {'_id': 0, 'samples': 1, 'accession': 1})))
#     .assign(samples=lambda d: d['samples'].map(len))
)
samples[:10]


,accession,samples
0,GSE1000,"[GSM15785, GSM15786, GSM15787, GSM15788, GSM15..."
1,GSE10000,"[GSM44658, GSM44659, GSM44660, GSM44661, GSM44..."
2,GSE10001,"[GSM252722, GSM252723, GSM252724, GSM252725, G..."
3,GSE10002,"[GSM252734, GSM252735, GSM252736, GSM252737, G..."
4,GSE10006,"[GSM252799, GSM252800, GSM252801, GSM252802, G..."
5,GSE10011,"[GSM252936, GSM252937, GSM252938, GSM252939, G..."
6,GSE10017,"[GSM253173, GSM253174]"
7,GSE10021,"[GSM253203, GSM253204, GSM253205, GSM253206, G..."
8,GSE10024,"[GSM253283, GSM253284, GSM253285, GSM253286, G..."
9,GSE10026,"[GSM252165, GSM252166, GSM252167, GSM252168, G..."


In [9]:
data = pd.merge(data, samples, left_on='series', right_on='accession', how='left')

In [10]:
data[:10]

,name,status,type,error,meta,platform,series,method,accession,samples
0,GSE53224_GPL570,done,preproc,None,"{'samples': ['GSM1287918', 'GSM1287919', 'GSM1...",GPL570,GSE53224,affymetrix,GSE53224,"[GSM1287918, GSM1287919, GSM1287920, GSM128792..."
1,GSE53183_GPL570,done,preproc,None,"{'samples': ['GSM1286097', 'GSM1286098', 'GSM1...",GPL570,GSE53183,affymetrix,GSE53183,"[GSM1286097, GSM1286098, GSM1286099, GSM128610..."
2,GSE53157_GPL570,done,preproc,None,"{'samples': ['GSM1283115', 'GSM1283116', 'GSM1...",GPL570,GSE53157,affymetrix,GSE53157,"[GSM1283115, GSM1283116, GSM1283117, GSM128311..."
3,GSE65721_GPL5082,skip,preproc,None,"{'samples': ['GSM1603234', 'GSM1603235', 'GSM1...",GPL5082,GSE65721,NaN,GSE65721,"[GSM1603234, GSM1603235, GSM1603236, GSM160323..."
4,GSE65721_GPL570,done,preproc,None,"{'samples': ['GSM1603354', 'GSM1603355', 'GSM1...",GPL570,GSE65721,affymetrix,GSE65721,"[GSM1603234, GSM1603235, GSM1603236, GSM160323..."
5,GSE53092_GPL570,done,preproc,None,"{'samples': ['GSM1282316', 'GSM1282317', 'GSM1...",GPL570,GSE53092,affymetrix,GSE53092,"[GSM1282316, GSM1282317, GSM1282318, GSM1282319]"
6,GSE65707_GPL570,done,preproc,None,"{'samples': ['GSM1603354', 'GSM1603355', 'GSM1...",GPL570,GSE65707,affymetrix,GSE65707,"[GSM1603354, GSM1603355, GSM1603356, GSM160335..."
7,GSE53059_GPL570,done,preproc,None,"{'samples': ['GSM1281432', 'GSM1281433', 'GSM1...",GPL570,GSE53059,affymetrix,GSE53059,"[GSM1281432, GSM1281433, GSM1281434, GSM128143..."
8,GSE53046_GPL570,done,preproc,None,"{'samples': ['GSM1281025', 'GSM1281026', 'GSM1...",GPL570,GSE53046,affymetrix,GSE53046,"[GSM1281025, GSM1281026, GSM1281027, GSM128102..."
9,GSE53012_GPL570,done,preproc,None,"{'samples': ['GSM1280329', 'GSM1280330', 'GSM1...",GPL570,GSE53012,affymetrix,GSE53012,"[GSM1280329, GSM1280330, GSM1280331, GSM128033..."


In [21]:
samples_lists = (
    data
    .query("status == 'done' and method == 'affymetrix'")
    .samples
    .dropna()
    .tolist()
)

samples_lists[:10]
samples_accessions = list({s for ss in samples_lists for s in ss})
# samples_accessions[:10]
len(samples_accessions)

203771

In [22]:
import json
with open('../data/geo-annotation/samples-affumetrix-preprocessed.json', 'w') as f:
    json.dump(samples_accessions, f)

In [96]:
names_mapping = """GPL570	Affymetrix HG-U133_Plus_2
GPL1261	Affymetrix Mouse430_2
GPL96	Affymetrix HG-U133A
GPL10558	Illumina HumanHT-12 V4.0
GPL6244	Affymetrix HuGene-1_0-st
GPL6947	Illumina HumanHT-12 V3.0
GPL6246	Affymetrix MoGene-1_0-st
GPL6480	Agilent 014850
GPL1355	Affymetrix Rat230_2
GPL4133	Agilent 014850
GPL571	Affymetrix HG-U133A_2
GPL97	Affymetrix HG-U133B
GPL5175	Affymetrix HuEx-1_0-st
GPL198	Affymetrix ATH1-121501
GPL6887	Illumina MouseWG-6 v2.0
GPL6885	Illumina MouseRef-8 v2.0
GPL6884	Illumina HumanWG-6 v3.0
GPL5188	Affymetrix HuEx-1_0-st
GPL81	Affymetrix MG_U74Av2
GPL7202	Agilent-014868"""

platform_names1 = (pd.DataFrame
 .from_records([line.split("\t") for line in names_mapping.split("\n")],
                columns=['accession', 'name'])
 )

platform_names1
# [(line.split("\t")) for line in names_mapping.split("\n")]

,accession,name
0,GPL570,Affymetrix HG-U133_Plus_2
1,GPL1261,Affymetrix Mouse430_2
2,GPL96,Affymetrix HG-U133A
3,GPL10558,Illumina HumanHT-12 V4.0
4,GPL6244,Affymetrix HuGene-1_0-st
5,GPL6947,Illumina HumanHT-12 V3.0
6,GPL6246,Affymetrix MoGene-1_0-st
7,GPL6480,Agilent 014850
8,GPL1355,Affymetrix Rat230_2
9,GPL4133,Agilent 014850


In [100]:
names2 = [
{"accession": "GPL887", "name": "Agilent-012097"},
{"accession": "GPL1708", "name": "Agilent-012391"},
{"accession": "GPL2879", "name": "Agilent-013282"},
{"accession": "GPL3921", "name": "Affymetrix HT_HG-U133A"},
{"accession": "GPL4091", "name": "Agilent-014693"},
{"accession": "GPL4093", "name": "Agilent-014698"},
{"accession": "GPL5477", "name": "Agilent-014950"},
{"accession": "GPL6848", "name": "Agilent-012391"},
{"accession": "GPL7264", "name": "Agilent-012097"},
{"accession": "GPL8321", "name": "Affymetrix Mouse430A_2"},
{"accession": "GPL9128", "name": "Agilent-014693"},
{"accession": "GPL10123", "name": "Agilent-022060"},
{"accession": "GPL10806", "name": "Agilent-023456"},
{"accession": "GPL11387", "name": "Agilent  180K v3.0"},
]

platform_names2 = (pd.DataFrame
 .from_records(names2,
                columns=['accession', 'name'])
 )

platform_names = pd.concat([platform_names1, platform_names2]).reset_index().drop('index', axis=1)#.drop('level_0', axis=1)
platform_names

,accession,name
0,GPL570,Affymetrix HG-U133_Plus_2
1,GPL1261,Affymetrix Mouse430_2
2,GPL96,Affymetrix HG-U133A
3,GPL10558,Illumina HumanHT-12 V4.0
4,GPL6244,Affymetrix HuGene-1_0-st
5,GPL6947,Illumina HumanHT-12 V3.0
6,GPL6246,Affymetrix MoGene-1_0-st
7,GPL6480,Agilent 014850
8,GPL1355,Affymetrix Rat230_2
9,GPL4133,Agilent 014850


In [105]:
_t = pd.merge(
    data
    [data.method.notnull()]
    .query("status == 'done' and type == 'preproc'")
    .groupby(['platform'])
    .samples
    .sum()
    .to_frame()
    .sort('samples', ascending=False),
    platform_names,
    left_index=True,
    right_on='accession',
    how='left'
).reset_index().drop('index', axis=1)[:20]
_t[['accession', 'name', 'samples']]

,accession,name,samples
0,GPL570,Affymetrix HG-U133_Plus_2,123491
1,GPL1261,Affymetrix Mouse430_2,51102
2,GPL96,Affymetrix HG-U133A,39525
3,GPL571,Affymetrix HG-U133A_2,14213
4,GPL8321,Affymetrix Mouse430A_2,7070
5,GPL6480,Agilent 014850,3892
6,GPL4133,Agilent 014850,3650
7,GPL3921,Affymetrix HT_HG-U133A,3545
8,GPL9128,Agilent-014693,3028
9,GPL4091,Agilent-014693,2974


In [79]:
unknown_platforms = pd.DataFrame(list(db.platforms.find({'accession': {'$in': _t[_t.name.isnull()].accession.tolist()}}, {'_id': 0, 'title': 1, 'accession' :1})))
unknown_platforms

,accession,title
0,GPL887,Agilent-012097 Human 1A Microarray (V2) G4110B...
1,GPL1708,Agilent-012391 Whole Human Genome Oligo Microa...
2,GPL2879,Agilent-013282 Human Genome CGH Microarray 44B...
3,GPL3921,[HT_HG-U133A] Affymetrix HT Human Genome U133A...
4,GPL4091,Agilent-014693 Human Genome CGH Microarray 244...
5,GPL4093,Agilent-014698 Human Genome CGH Microarray 105...
6,GPL5477,Agilent-014950 Human Genome CGH Microarray 4x4...
7,GPL6848,Agilent-012391 Whole Human Genome Oligo Microa...
8,GPL7264,Agilent-012097 Human 1A Microarray (V2) G4110B...
9,GPL8321,[Mouse430A_2] Affymetrix Mouse Genome 430A 2.0...


In [78]:
db.platforms.count()

23231

In [109]:
', '.join(_t[_t.accession.isin(FRMA_PLATFORMS)].accession)


'GPL570, GPL1261, GPL96, GPL571, GPL8321, GPL3921'

In [110]:

', '.join(_t[_t.accession.isin(AGILENT_PLATFORMS)].accession)

'GPL6480, GPL4133, GPL9128, GPL4091, GPL1708, GPL7264, GPL887, GPL6848, GPL2879, GPL5477, GPL10123, GPL4093, GPL10806, GPL11387'

In [113]:
data.query('method == "agilent" and status == "done"')

,name,status,type,error,meta,platform,series,method,accession,samples
3816,GSE26863_GPL9128,done,preproc,None,"{'platform': 'GPL9128', 'accession': 'GSE26863...",GPL9128,GSE26863,agilent,GSE26863,501
3929,GSE23720_GPL9128,done,preproc,None,"{'platform': 'GPL9128', 'accession': 'GSE23720...",GPL9128,GSE23720,agilent,GSE23720,370
4321,GSE9015_GPL4091,done,preproc,None,"{'platform': 'GPL4091', 'accession': 'GSE9015'...",GPL4091,GSE9015,agilent,GSE9015,8
11750,GSE21468_GPL10123,done,preproc,None,"{'platform': 'GPL10123', 'accession': 'GSE2146...",GPL10123,GSE21468,agilent,GSE21468,74
11751,GSE23065_GPL10123,done,preproc,None,"{'platform': 'GPL10123', 'accession': 'GSE2306...",GPL10123,GSE23065,agilent,GSE23065,2
11753,GSE24602_GPL10123,done,preproc,None,"{'platform': 'GPL10123', 'accession': 'GSE2460...",GPL10123,GSE24602,agilent,GSE24602,107
11754,GSE25228_GPL10123,done,preproc,None,"{'platform': 'GPL10123', 'accession': 'GSE2522...",GPL10123,GSE25228,agilent,GSE25228,17
11755,GSE25411_GPL10123,done,preproc,None,"{'platform': 'GPL10123', 'accession': 'GSE2541...",GPL10123,GSE25411,agilent,GSE25411,1
11756,GSE26715_GPL10123,done,preproc,None,"{'platform': 'GPL10123', 'accession': 'GSE2671...",GPL10123,GSE26715,agilent,GSE26715,2
11757,GSE27671_GPL10123,done,preproc,None,"{'platform': 'GPL10123', 'accession': 'GSE2767...",GPL10123,GSE27671,agilent,GSE27671,13


In [116]:
data.query("type == 'preproc' and status == 'done'").shape[0]

9316

In [117]:
data.query("type == 'preproc' and status == 'done'").samples.sum()

270053.0